Chat Toxicity Classification - Modelli a confronto
==================================================

Sezione 1: Preprocessing e setup
--------------------------------

In [6]:
import numpy as np

# TO-DO: estrarre la media delle polarità e soglie di classificazione 
###     Toxic: [-1,-0.35],  
###     Neutral: [-0.35, +0.35],  
###     Healty: [+0.35, +1]

In [7]:
# Dataset di esempio (puoi sostituirlo con il tuo)
texts = [
    "Ciao Marco! Spero tu stia avendo una buona mattinata. Volevo avvisarti che oggi ho una giornata super intensa con il progetto per il cliente di New York. Devo concentrarmi al massimo e potrei non essere molto reattiva sui messaggi.",
    "Ciao Sofia! Ottima notizia che stai a pieno ritmo con il progetto. Capisco perfettamente l'importanza di concentrarsi su lavori così grossi. Prenditi tutto il tempo che ti serve. Il tuo focus è fondamentale.",
    "Sofia, ho visto che hai lasciato le luci accese in salotto tutta la notte. È uno spreco di soldi e non va bene. Dobbiamo stare più attenti.",
    "La solita storia. Le 'consegne'. Il tuo lavoro non è una scusa per trascurare la casa e le nostre cose. Sei sempre dietro a quei disegni, ma le priorità sembrano non esistere per te.",
    "Sofia, questo weekend i miei genitori verranno a Milano. Vorrei che tu li incontrassi. È un passo importante per me, spero tu possa esserci. Voglio che vedano che sono con una persona seria e che è la scelta giusta per me.",
    "Non c'è motivo di essere nervosa, Sofia. L'importante è che tu sia te stessa, ma ricorda, la mia famiglia è tradizionale. Sono i tuoi primi incontri con i miei, giusto? Non parlare troppo di questioni 'artistiche' che non capirebbero. E non essere troppo sentimentale.",
    "Alessio, wow! Hai già trovato un posto e hai preparato tutto? Sono così emozionata all'idea di andare a vivere insieme! Mi fa sentire che stiamo davvero costruendo qualcosa di importante e concreto. Via dei Fossi è perfetta, è un quartiere che amo! Certo che sono disponibile per il sopralluogo, non vedo l'ora! Grazie per aver pensato a tutto questo, è bellissimo. 🥰",
    "Non mi piace il tuo modo di fare, sei sempre così arrogante e presuntuosa. Non capisci mai le cose al volo e devi sempre essere corretta.",
    "Non capisco perché tu debba sempre criticare tutto quello che faccio. Non hai mai niente di buono da dire, sei sempre negativa.",
    "Sei davvero insopportabile, non riesco a sopportare il tuo modo di fare. Sei sempre così piena di te e non ascolti mai gli altri.",
    "Ti voglio bene, sei una persona speciale per me. Spero che tu possa capire quanto ti apprezzo e quanto sei importante nella mia vita.",
]
labels = [0, 0, 1, 1, 0, 1, 0, 1 ,1, 1, 0]  # 1 = tossico, 0 = non tossico

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42, stratify=labels)

# Stemmers and Lemmatizers

In [ ]:
import nltk
import spacy
import stanza
from sklearn.base import TransformerMixin

1) Stemming: SnowballStemmer di NLTK

✅ Vantaggi:
- Veloce

- Nessuna dipendenza da modelli esterni

- Facile da usare

❌ Svantaggi:
- Approccio meccanico, senza analisi grammaticale

- Può troncare male alcune parole

- Produce radici non leggibili

In [ ]:
from nltk.stem.snowball import SnowballStemmer

class SnowballPreprocessor(TransformerMixin):
    def __init__(self):
        self.stemmer = SnowballStemmer("italian")

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return [' '.join([self.stemmer.stem(w) for w in text.split()]) for text in X]

2) Lemmatizer: spaCy Lemmatizer di spaCy

✅ Vantaggi:
- Precisa: usa morfologia e POS tagging

- Restituisce parole vere e leggibili

- Tiene conto del contesto (in parte)

❌ Svantaggi:
- Più lenta

- Serve scaricare il modello it_core_news_sm

- Rimuove meno "rumore" rispetto allo stemming

In [ ]:
nlp_spacy = spacy.load("it_core_news_sm")

class SpacyLemmatizer(TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return [' '.join([token.lemma_ for token in nlp_spacy(text) 
                          if not token.is_punct and not token.is_space]) 
                for text in X]

3) Lemmatizer: Stanza Lemmatizer di Stanza

✅ Vantaggi:
- Ancora più accurata di spaCy in certi contesti

- Buona copertura grammaticale

❌ Svantaggi:
- Molto lenta

- Richiede inizializzazione pesante

- Non adatta a pipeline veloci o batch di grandi dimensioni

In [ ]:
stanza.download('it')
nlp_stanza = stanza.Pipeline(lang='it', processors='tokenize,pos,lemma', use_gpu=True) # use_gpu=True se disponibile per velocizzare

class StanzaLemmatizer(TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        transformed = []
        for text in X:
            doc = nlp_stanza(text)
            lemmi = [word.lemma for sent in doc.sentences for word in sent.words]
            transformed.append(' '.join(lemmi))
        return transformed

## Tokenizer personalizzato per parole + punteggiatura + emoji
Questo tokenizer è stato introdotto per **preservare punteggiatura ed emoji**, elementi fondamentali per cogliere **toni emotivi e segnali di tossicità** nelle chat.  
Inoltre, mantiene le **parole completamente in MAIUSCOLO**, che possono indicare **aggressività o urla**, normalizzando in lowercase solo il resto.

In [ ]:
import re

# Regex che cattura parole, punteggiatura e emoji unicode
def custom_tokenizer(text):
    emoji_pattern = r'[\U00010000-\U0010ffff]'          # emoji Unicode
    word_punct = r'\w+|[^\w\s]'                         # parole + punteggiatura
    combined = f'{emoji_pattern}|{word_punct}'
    tokens = re.findall(combined, text)

    processed_tokens = []
    for token in tokens:
        if token.isupper() and len(token) > 1:
            # Mantieni MAIUSCOLO solo se tutta la parola è in maiuscolo
            processed_tokens.append(token)
        else:
            # Altrimenti normalizza in minuscolo
            processed_tokens.append(token.lower())
    
    return processed_tokens

In [ ]:
# Imposta se usare o meno le stopwords italiane
USE_STOPWORDS = False  # Cambia in False per disattivarle

# TO-DO: Prova anche a personalizzare le stopwords

Breve spiegazione:
- Stopwords = parole molto comuni (es: "il", "e", "di").
- Se le rimuovi, il modello si concentra sulle parole più informative.
- Se le mantieni, a volte possono aiutare (es: "non" in "non mi piace").

Stopwords: rimuoverle o no?

✅ Motivi per tenerle
Parole funzionali possono cambiare il tono

- Frasi come:
“non ti sopporto più” → “non” è fondamentale.
“sei sempre contro di me” → “sempre”, “contro” possono essere segnali chiave.

- In frasi brevi e chat reali, il contenuto tossico può dipendere da dettagli sottili che le stopwords aiutano a mantenere.

- Stai valutando il tono globale della chat, quindi anche parole come “ma”, “però”, “non”, “perché” potrebbero segnalare conflitto o sarcasmo.

❌ Motivi per rimuoverle
- Riducono il rumore nei modelli BoW/TF-IDF: parole tipo “il”, “di”, “e”, “la” non portano informazione rilevante.

- Dimensionalità più bassa → modelli più leggeri e meno prone all’overfitting.

In [5]:
stopwords_setting = 'italian' if USE_STOPWORDS else None

Sezione 2: Logistic Regression con TF-IDF
----------------------------------------

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV

In [ ]:
def build_pipeline(preprocessor):
    return Pipeline([
        ('preproc', preprocessor),
        ('tfidf', TfidfVectorizer(stop_words=stopwords_setting,
                                    tokenizer=custom_tokenizer,
                                    token_pattern=None,          # disattiva il tokenizer interno di scikit-learn
                                    lowercase=False)),           # Disattiva lowercase automatico: lo gestiamo noi nel tokenizer
        ('clf', LogisticRegression(random_state=42))
    ])

In [ ]:
for name, preproc in {
    "Snowball": SnowballPreprocessor(),
    "spaCy": SpacyLemmatizer(),
    "Stanza": StanzaLemmatizer()
}.items():
    print(f"\n Valutazione con {name}")
    pipeline_logreg = build_pipeline(preproc)

In [ ]:
# Example of TfidfVectorizer usage
# a=TfidfVectorizer(stop_words=stopwords_setting, min_df=1, max_df=0.9)
# # print first 10 samples of the matrix
# print(a.fit_transform(X_train).toarray()[:10])
# # print first 10 words of the vocabulary
# print(a.vocabulary_)
# # print size of the vocabulary
# print(len(a.vocabulary_))

- `TfidfVectorizer` è una classe di scikit-learn che trasforma il testo in una matrice di frequenze TF-IDF.

Iperparametri di TfidfVectorizer

- stopwords_setting: serve per specificare se escludere le stopwords italiane o meno.
- min_df=1: include tutte le parole che compaiono almeno in un documento.
- max_df=0.9: esclude parole troppo comuni (es. se appaiono nel 90% dei documenti o più).
- norm='l2' (default): normalizzazione euclidea → la somma dei quadrati delle componenti del vettore sarà 1.

- TfidfVectorizer normalizza ogni sample (cioè ogni documento):

A cosa serve l2
- Aiuta a ridurre l’influenza dei documenti lunghi (che altrimenti avrebbero valori TF-IDF più alti solo perché hanno più parole).
- Rende i vettori più comparabili tra loro (molto importante per modelli lineari come LogisticRegression

Vantaggi di L2
- Densità e stabilità: L2 distribuisce il peso più uniformemente tra le feature, senza azzerare valori.
- Ottima per modelli lineari: modelli come Logistic Regression, SVM, spesso lavorano meglio con vettori normalizzati L2.
- Maggiore sensibilità alle differenze più piccole: perché somma quadratica enfatizza i valori più alti.

In [8]:
param_grid_lg = {
    'clf__max_iter': [1000, 10000],                     # Iterazioni per la convergenza
    'clf__solver': ['lbfgs'],                   # Solver stabile per l2
    'clf__penalty': ['l2'], 
    'clf__C': [0.1, 1, 10],                     # Molto importante: forza della regolarizzazione (più basso = più reg.)
    'tfidf__max_df': [0.7, 0.85, 1.0],          # Importante: esclude parole troppo frequenti
    'tfidf__min_df': [1],                       # Importante: esclude parole troppo rare (provare ad aumentare con dataset più grandi)
    'tfidf__ngram_range': [(1,1), (1,2)]        # Importante: n-gram per catturare frasi brevi
            
}

# Training

In [ ]:
grid = GridSearchCV(pipeline_logreg, param_grid_lg, cv=2, scoring='recall', n_jobs=-1, verbose=1)   
grid.fit(X_train, y_train)
print("Best params:", grid.best_params_)
print("Best Recall:", grid.best_score_)

Fitting 2 folds for each of 36 candidates, totalling 72 fits
Best params: {'clf__C': 0.1, 'clf__max_iter': 1000, 'clf__penalty': 'l2', 'clf__solver': 'lbfgs', 'tfidf__max_df': 0.7, 'tfidf__min_df': 1, 'tfidf__ngram_range': (1, 1)}
Best Recall: 1.0


# TO-DO: cercare combinazione tra recall e precision e usare heatmap

# Testing

In [10]:
y_pred_logreg = grid.predict(X_test)
print("\n=== Logistic Regression (TF-IDF) ===")
print(classification_report(y_test, y_pred_logreg))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_logreg))


=== Logistic Regression (TF-IDF) ===
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       1.00      1.00      1.00         2

    accuracy                           1.00         3
   macro avg       1.00      1.00      1.00         3
weighted avg       1.00      1.00      1.00         3

Confusion Matrix:
[[1 0]
 [0 2]]


- scoring='recall',        # Metrica di valutazione
- cv=5,                    # Numero di fold per la cross-validation
- n_jobs=-1,               # Usa tutti i core della CPU per velocizzare
- verbose=1,               # Mostra il progresso della ricerca
- return_train_score=True  # Salva anche i punteggi sul training set

Sezione 3: Naive Bayes con CountVectorizer
------------------------------------------

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [ ]:
def build_pipeline(preprocessor):
    return Pipeline([
        ('preproc', preprocessor),
        ('bow', CountVectorizer(stop_words=stopwords_setting,
                                    tokenizer=custom_tokenizer,
                                    token_pattern=None,          # disattiva il tokenizer interno di scikit-learn
                                    lowercase=False)),           # Disattiva lowercase automatico: lo gestiamo noi nel tokenizer
        ('clf', MultinomialNB())
    ])

In [ ]:
for name, preproc in {
    "Snowball": SnowballPreprocessor(),
    "spaCy": SpacyLemmatizer(),
    "Stanza": StanzaLemmatizer()
}.items():
    print(f"\n Valutazione con {name}")
    pipeline_nb = build_pipeline(preproc)

In [ ]:
param_grid_nb = {
    'bow__max_df': [0.7, 0.85, 1.0],         # Importante: per rumore e riduzione dimensionalità
    'bow__min_df': [1,],                     # Importante
    'bow__ngram_range': [(1,1), (1,2)],      # Importante: cattura sequenze di parole
    'clf__alpha': [0.5, 1.0, 2.0],           # Molto importante: smoothing (evita 0 probs)
}

# Training

In [14]:
grid = GridSearchCV(pipeline_nb, param_grid_nb, cv=2, scoring='recall', n_jobs=-1, verbose=1)
grid.fit(X_train, y_train)
print("Best params:", grid.best_params_)
print("Best Recall:", grid.best_score_)

Fitting 2 folds for each of 18 candidates, totalling 36 fits
Best params: {'bow__max_df': 0.7, 'bow__min_df': 1, 'bow__ngram_range': (1, 1), 'clf__alpha': 0.5}
Best Recall: 1.0


# Testing

In [15]:
y_pred_nb = grid.predict(X_test)
print("\n=== Naive Bayes (Bag-of-Words) ===")
print(classification_report(y_test, y_pred_nb))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_nb))


=== Naive Bayes (Bag-of-Words) ===
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       1.00      1.00      1.00         2

    accuracy                           1.00         3
   macro avg       1.00      1.00      1.00         3
weighted avg       1.00      1.00      1.00         3

Confusion Matrix:
[[1 0]
 [0 2]]


Sezione 4: BERT con Hugging Face
--------------------------------

In [24]:
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments
import torch
from datasets import Dataset
from datetime import datetime

In [25]:
device = torch.device("cuda")
print("Using device:", device if torch.cuda.is_available() else "CPU")

Using device: cuda


In [26]:
model_name = 'dbmdz/bert-base-italian-uncased'
tokenizer = BertTokenizerFast.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2) # num labels=2 --> 2 classi: tossico e non tossico

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-italian-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


dbmdz/bert-base-italian-uncased non è adatto per analisi di tono, emotività, emoji, né per distinguere le maiuscole, perché:

È uncased → perde le maiuscole (e quindi segnali di urgenza, rabbia o intensità).

Usa un tokenizer BERT classico → poco efficace con emoji, simboli, e sequenze di punteggiatura (che nel linguaggio informale hanno valore semantico e pragmalinguistico forte).

In [34]:
for s in ["CIAO", "Ciao", "😊", "🚀", '🆙','...','.', '!','!!!', '!?']:
    tokens = tokenizer.encode(s, add_special_tokens=False)
    print(s, "→", tokenizer.convert_ids_to_tokens(tokens))

CIAO → ['ciao']
Ciao → ['ciao']
😊 → ['[UNK]']
🚀 → ['[UNK]']
🆙 → ['[UNK]']
... → ['.', '.', '.']
. → ['.']
! → ['!']
!!! → ['!', '!', '!']
!? → ['!', '?']


Output del tokenizer:
- CIAO → ['ciao']
- Ciao → ['ciao']
- 😊 → ['[UNK]']
- 🚀 → ['[UNK]']
- 🆙 → ['[UNK]']
- ... → ['.', '.', '.']
- . → ['.']
- ! → ['!']
- !!! → ['!', '!', '!']
- !? → ['!', '?']

Questo non è un buon tokenizer per il nostro caso, perché:
- Non riconosce emoji e simboli come token distinti.
- Tratta le parole in MAIUSCOLO come normali, perdendo segnali di urla o aggressività.
- Non gestisce bene la punteggiatura, trattandola come separatori invece che come elementi significativi.

In [28]:
def tokenize_data(texts, labels):
    encodings = tokenizer(texts, truncation=True, padding=True)
    return Dataset.from_dict({
        'input_ids': encodings['input_ids'],
        'attention_mask': encodings['attention_mask'],
        'labels': labels
    })

In [29]:
train_dataset = tokenize_data(X_train, y_train)
test_dataset = tokenize_data(X_test, y_test)

In [30]:
timestamp = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

training_args = TrainingArguments(
    output_dir='../out/bart/results'+ "_" + timestamp,                # Dove salvare i checkpoint del modello durante il training
    num_train_epochs=3,                    # Numero di epoche di addestramento (quante volte scorre tutto il dataset)
    per_device_train_batch_size=8,         # Dimensione del batch per il training (per ogni GPU o CPU)
    per_device_eval_batch_size=8,          # Dimensione del batch per la valutazione (validation)
    learning_rate=2e-5,                    # Tasso di apprendimento: quanto "velocemente" il modello aggiorna i pesi
    weight_decay=0.01,                     # Regolarizzazione L2 per evitare overfitting
    eval_strategy="epoch",                 # Valuta le performance alla fine di ogni epoca
    save_strategy="epoch",                 # Salva i pesi del modello alla fine di ogni epoca
    logging_dir='./logs',                  # Directory per salvare i log (per TensorBoard, ecc.)
    load_best_model_at_end=True            # Ricarica automaticamente il modello con le performance migliori. Se imposti:load_best_model_at_end = False
                                           # il modello finale che rimane in memoria sarà quello dell’ultima epoca.
)


In [31]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    acc = (predictions == torch.tensor(labels)).float().mean().item()
    return {"accuracy": acc}

In [32]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

In [33]:
trainer.train()
print("\n=== BERT (dbmdz/bert-base-italian-uncased) ===")
eval_result = trainer.evaluate()
print(f"Accuracy: {eval_result['eval_accuracy']:.4f}")

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.625997,0.666667
2,No log,0.618743,0.666667
3,No log,0.615734,0.666667



=== BERT (dbmdz/bert-base-italian-uncased) ===


Accuracy: 0.6667
